# Forward Model Selection

In [18]:
import statsmodels.api as sm

In [19]:
y = data['log_price']

X_ = data.drop(['price','log_price'], axis=1).astype('float64')

# Define the feature set X.
X = pd.concat([X_], axis=1).astype(float)
Xcons = X
Xcons = sm.add_constant(Xcons)

In [20]:
Xcons

,const,id,host_id,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,...,Williamsburg,Willowbrook,Windsor Terrace,Woodhaven,Woodlawn,Woodrow,Woodside,Entire home/apt,Private room,Shared room
0,1.0,2539.0,2787.0,40.64749,-73.97237,1.0,9.0,0.210000,6.0,365.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,2595.0,2845.0,40.75362,-73.98377,1.0,45.0,0.380000,2.0,355.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,3647.0,4632.0,40.80902,-73.94190,3.0,0.0,-0.731736,1.0,365.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,3831.0,4869.0,40.68514,-73.95976,1.0,270.0,4.640000,1.0,194.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,5022.0,7192.0,40.79851,-73.94399,10.0,9.0,0.100000,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,1.0,36484665.0,8232441.0,40.67853,-73.94995,2.0,0.0,1.746913,2.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
48891,1.0,36485057.0,6570630.0,40.70184,-73.93317,4.0,0.0,1.714120,2.0,36.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
48892,1.0,36485431.0,23492952.0,40.81475,-73.94867,10.0,0.0,1.798986,1.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48893,1.0,36485609.0,30985759.0,40.75751,-73.99112,1.0,0.0,1.846428,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate rsquared
    if 'const' not in feature_set:
        feature_set = feature_set + ('const',) # mannually add intercept
    model = sm.OLS(y,Xcons[list(feature_set)])
    regr = model.fit()
    rsquared = regr.rsquared
    return {"model":regr, "rsquared":rsquared}

In [26]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]

    tic = time.time()

    results = []

    for p in remaining_predictors:
        results.append(processSubset(tuple(predictors+[p])))

    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)

    # Choose the model with the highest rsquared
    best_model = models.loc[models['rsquared'].argmax()]

    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")

    # Return the best model, along with some other useful information about the model
    return best_model

In [27]:
import time
import itertools

In [28]:
models_fwd = pd.DataFrame(columns=["rsquared", "model"])

tic = time.time()
predictors = []

for i in range(1,15):
    models_fwd.loc[i] = forward(predictors)
    predictors = models_fwd.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  239 models on 1 predictors in 5.279508829116821 seconds.
Processed  238 models on 3 predictors in 5.345500469207764 seconds.
Processed  237 models on 4 predictors in 5.929432153701782 seconds.
Processed  236 models on 5 predictors in 7.096207857131958 seconds.
Processed  235 models on 6 predictors in 8.239491939544678 seconds.
Processed  234 models on 7 predictors in 10.697118520736694 seconds.
Processed  233 models on 8 predictors in 11.080331325531006 seconds.
Processed  232 models on 9 predictors in 11.874258279800415 seconds.
Processed  231 models on 10 predictors in 13.878157377243042 seconds.
Processed  230 models on 11 predictors in 11.923563957214355 seconds.
Processed  229 models on 12 predictors in 12.157837152481079 seconds.
Processed  228 models on 13 predictors in 15.752790212631226 seconds.
Processed  227 models on 14 predictors in 15.74014925956726 seconds.
Processed  226 models on 15 predictors in 32.930081367492676 seconds.
Total elapsed time: 173.3146862983

In [30]:
print(models_fwd.loc[14, "model"].summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.521
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     3794.
Date:                Sun, 03 Dec 2023   Prob (F-statistic):               0.00
Time:                        14:45:13   Log-Likelihood:                -33823.
No. Observations:               48884   AIC:                         6.768e+04
Df Residuals:                   48869   BIC:                         6.781e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Entire home/apt       

# Forward Selection MSE 

In [36]:
from sklearn.model_selection import train_test_split
import sklearn.model_selection as skm

In [37]:
(X_train_mod, X_test_mod, y_train_mod, y_test_mod) = skm.train_test_split(data[['Entire home/apt', 'nearby_subway_line_counts', 'Manhattan', 'availability_365', 'Williamsburg', 'Shared room', 'Harlem', 'Washington Heights', 'longitude', 'minimum_nights', 'Midtown', 'Greenpoint', 'Staten Island', 'number_of_reviews']],
                                                          data['log_price'],
                                                          test_size=0.2,
                                                          random_state=0)

In [38]:
X_train_mod.head()

,Entire home/apt,nearby_subway_line_counts,Manhattan,availability_365,Williamsburg,Shared room,Harlem,Washington Heights,longitude,minimum_nights,Midtown,Greenpoint,Staten Island,number_of_reviews
2201,0.0,0.0,1.0,320.0,0.0,0.0,0.0,0.0,-73.97907,5.0,0.0,0.0,0.0,35.0
47843,1.0,3.0,0.0,117.0,0.0,0.0,0.0,0.0,-73.98258,3.0,0.0,0.0,0.0,1.0
30511,0.0,1.0,0.0,89.0,0.0,0.0,0.0,0.0,-73.94080,4.0,0.0,0.0,0.0,2.0
44290,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,-73.91165,1.0,0.0,0.0,0.0,7.0
47654,0.0,2.0,0.0,20.0,0.0,0.0,0.0,0.0,-74.02001,2.0,0.0,0.0,0.0,0.0


In [39]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import math
from sklearn.metrics import mean_squared_error

In [41]:
# Linear Regression Train
lm = LinearRegression()
lm.fit(X_train_mod, y_train_mod)

LinearRegression()

In [44]:
y_lm_pred_train = lm.predict(X_train_mod)
train_mse = mean_squared_error(y_train_mod, y_lm_pred_train)
print("Training Error (MSE) using the 15 selected predictors:", train_mse)

Training Error (MSE) using the 15 selected predictors: 0.23284616776686481


In [45]:
y_lm_pred_test = lm.predict(X_test_mod)
test_mse = mean_squared_error(y_test_mod, y_lm_pred_test)
print("Test Error (MSE) using the 15 selected predictors:", test_mse)

Test Error (MSE) using the 15 selected predictors: 0.23675961752170155
